In [1]:
from pyomo.environ import *

In [2]:
# PU calcualtions
sbase = 100;
#Line impeadance PU
x12 = 0.2   
x32 = 0.25
x13 = 0.4
#slack bus 
delta3 = 0

In [3]:
#defining model
model = AbstractModel()
#Power Generation
model.pg1 = Var(bounds=(0,0.65))
model.pg2 = Var(bounds=(0,1.00))

#power being sent from from one bus to the next
model.p12 = Var(bounds=(-1,1))
model.p13 = Var(bounds=(-1,1))
model.p32 = Var(bounds=(-1,1))

#voltage angle @ each bus 
model.delta1 = Var()
model.delta2 = Var()

In [4]:
'''   Constraints    '''
#active flow calculation between each pair of connected buses
def con_rule1(model):
    return model.pg1  == model.p12 + model.p13
model.con1 = Constraint(rule=con_rule1)

def con_rule2(model):
    return model.p12  + model.p32 ==  1
model.con2 = Constraint(rule=con_rule2)

def con_rule3(model):
    return model.pg2 + model.p13 == model.p32   
model.con3 = Constraint(rule=con_rule3)

#nodal active power balance in each bus
def con_rule4(model):
    return model.p12 == (model.delta1 - model.delta2)/x12
model.con4 = Constraint(rule=con_rule4)

def con_rule5(model):
    return model.p13 == (model.delta1 - delta3)/x13
model.conf = Constraint(rule=con_rule5)

def con_rule6(model):
    return model.p32 == (delta3 - model.delta2)/x32
model.con6 = Constraint(rule=con_rule6)


In [5]:

#Objective Function, trying to minimize the cost.
def obj_rule(model):
    return 10*model.pg1+11*model.pg2
model.OF = Objective(rule=obj_rule,sense=minimize)



In [6]:

opt = SolverFactory('glpk')
instance = model.create_instance()
results = opt.solve(instance) # solves and updates instance
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  result.solver.status)

feasible


In [8]:
print("DC 3 Bus PowerFlow")
print("Example 6.3 taken from Power System Optimization Modeling in GAMS")
print("Report")
print('Power Generated at Bus 1 = '+str(round(sbase*value(instance.pg1),2))+" MW")
print('Power Generated at Bus 2 = '+str(round(sbase*value(instance.pg2),2))+" MW")
print("--------------------------------------")
print('Power Flowing from bus 1 to 2 = '+str(round(sbase*value(instance.p12),2))+" MW")
print('Power Flowing from bus 1 to 3 = '+str(round(sbase*value(instance.p13),2))+" MW")
print('Power Flowing from bus 3 to 2 = '+str(round(sbase*value(instance.p32),2))+" MW")
print("--------------------------------------")
print('Delta 1 = '+str(round(value(instance.delta1),2)))
print('Delta 2 = '+str(round(value(instance.delta2),2)))
print('Delta 3 = '+str(delta3))
print("--------------------------------------")
print('cost = '+ str(round(sbase*value(instance.OF),2))+' €/Hour')

DC 3 Bus PowerFlow
Example 6.3 taken from Power System Optimization Modeling in GAMS
Report
Power Generated at Bus 1 = 65.0 MW
Power Generated at Bus 2 = 35.0 MW
--------------------------------------
Power Flowing from bus 1 to 2 = 60.0 MW
Power Flowing from bus 1 to 3 = 5.0 MW
Power Flowing from bus 3 to 2 = 40.0 MW
--------------------------------------
Delta 1 = 0.02
Delta 2 = -0.1
Delta 3 = 0
--------------------------------------
cost = 1035.0 €/Hour
